In [1]:
import numpy as np 
import pandas as pd 
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import requests
import subprocess
import re
from tqdm import tqdm
tqdm.pandas()

import os
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from logger import logger

In [2]:
# # Initialize WebDriver
# url = 'https://www.imdb.com/title/tt10888594/?ref_=sr_i_1'
# driver = webdriver.Chrome()
# driver.get(url)
# time.sleep(3)

# # Get reviews link
# click_reviews = driver.find_element(By.XPATH, "//div[@data-testid='reviews-header']")
# reviews_link = click_reviews.find_element(By.TAG_NAME, 'a').get_attribute('href')
# driver.get(reviews_link)

# # Extract total reviews count
# reviews_element = driver.find_element(By.XPATH, "//div[@data-testid='tturv-total-reviews']")
# reviews_text = reviews_element.text
# reviews_count = int(re.sub(r"[^\d]", "", reviews_text))
# print(f"Total reviews: {reviews_count}")

# # Function to click 'Show All' button
# def click_all_button():
#     try:
#         span_element = driver.find_element(By.XPATH, "//span[contains(@class, 'chained-see-more-button')]")
#         button = span_element.find_element(By.TAG_NAME, "button")
#         # if button:
#         ActionChains(driver).move_to_element(button).click().perform()
#             # logger.info("'Show All' button clicked successfully.")
#     except Exception as e:
#         # logger.info(f"Error clicking button: {e}")
#         pass

# # Load all reviews
# max_wait_time = 600  # Max wait time (seconds)
# polling_interval = 1.5  # Time between checks
# target_count = reviews_count  # Expected number of reviews
# start_time = time.time()

# click_all_button()

# while True:
#     articles = driver.find_elements(By.TAG_NAME, "article")
#     current_count = len(articles)
#     print(f"Loaded articles: {current_count}")
    
#     if current_count >= target_count:
#         logger.info("All articles loaded!")
#         break
    
#     if time.time() - start_time > max_wait_time:
#         print("Timeout reached. Extracting available articles.")
#         break
    
#     if articles:
#         last_article = articles[-1]
#         driver.execute_script("arguments[0].scrollIntoView();", last_article)

#     click_all_button()
#     time.sleep(polling_interval)
    


# # Extract review details
# sample_articles = articles[:200]  # Limit to 100 articles
# review_stars, max_stars, review_titles, review_texts = [], [], [], []

# for article in tqdm(sample_articles):
#     try:
#         review_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--rating').text
#     except NoSuchElementException:
#         review_star = None
#     review_stars.append(review_star)
    
#     try:
#         max_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--maxRating').text[1:]
#     except NoSuchElementException:
#         max_star = None
#     max_stars.append(max_star)
    
#     try:
#         review_title = article.find_element(By.CLASS_NAME, 'ipc-title__text').text
#     except NoSuchElementException:
#         review_title = None
#     review_titles.append(review_title)
    
#     try:
#         review_text = article.find_element(By.CLASS_NAME, "ipc-overflowText--children").text
#     except NoSuchElementException:
#         review_text = None
#     review_texts.append(review_text)
    
# df = pd.DataFrame({
#     "review_star": review_stars,
#     "max_star": max_stars,
#     "review_title": review_titles,
#     "review_text": review_texts
# })

# # Close the WebDriver
# driver.quit()

# print("Scraping completed!")

In [3]:
# import time
# import re
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.common.exceptions import NoSuchElementException
# from tqdm import tqdm

# class IMDbReviewScraper:
#     """A scraper to extract user reviews from multiple IMDb movie pages."""

#     def __init__(self, movie_urls):
#         """
#         Initializes the IMDbReviewScraper.

#         Args:
#             movie_urls (list): List of IMDb movie URLs to scrape.
#         """
#         self.movie_urls = movie_urls
#         self.driver = webdriver.Chrome()
#         self.all_reviews = []

#     def get_reviews_page(self, url):
#         """
#         Navigates to the IMDb movie reviews page.

#         Args:
#             url (str): IMDb movie URL.

#         Returns:
#             str: The reviews page URL if found, else None.
#         """
#         self.driver.get(url)
#         time.sleep(3)

#         try:
#             click_reviews = self.driver.find_element(By.XPATH, "//div[@data-testid='reviews-header']")
#             reviews_link = click_reviews.find_element(By.TAG_NAME, 'a').get_attribute('href')
#             return reviews_link
#         except NoSuchElementException:
#             logger.info(f"Reviews link not found for {url}")
#             return None

#     def get_total_reviews_count(self):
#         """
#         Extracts the total number of reviews available for the movie.

#         Returns:
#             int: Total number of reviews if found, else 0.
#         """
#         try:
#             reviews_element = self.driver.find_element(By.XPATH, "//div[@data-testid='tturv-total-reviews']")
#             reviews_text = reviews_element.text
#             return int(re.sub(r"[^\d]", "", reviews_text))
#         except NoSuchElementException:
#             logger.info("Could not retrieve total reviews count.")
#             return 0

#     def click_show_all_button(self):
#         """
#         Clicks the 'Show All' button to load more reviews.
#         """
#         try:
#             span_element = self.driver.find_element(By.XPATH, "//span[contains(@class, 'chained-see-more-button')]")
#             button = span_element.find_element(By.TAG_NAME, "button")
#             if button.text == 'All':
#                 logger.info('clicking All')
#                 ActionChains(self.driver).move_to_element(button).click().perform()
#         except Exception:
#             pass

#     def load_all_reviews(self, reviews_count, max_wait_time=600, polling_interval=1.5):
#         """
#         Loads all available reviews by scrolling and clicking 'Show All'.

#         Args:
#             reviews_count (int): Expected number of reviews.
#             max_wait_time (int, optional): Maximum time to wait for loading. Defaults to 600 seconds.
#             polling_interval (float, optional): Time delay between scrolls. Defaults to 1.5 seconds.

#         Returns:
#             list: List of WebElement articles containing reviews.
#         """
#         start_time = time.time()
#         self.click_show_all_button()

#         while True:
#             articles = self.driver.find_elements(By.TAG_NAME, "article")
#             current_count = len(articles)
#             # print(f"Loaded articles: {current_count}")

#             if current_count >= reviews_count or time.time() - start_time > max_wait_time:
#                 logger.info(f"loaded {current_count} review objectss")
#                 break

#             if articles:
#                 last_article = articles[-1]
#                 self.driver.execute_script("arguments[0].scrollIntoView();", last_article)

#             self.click_show_all_button()
#             time.sleep(polling_interval)

#         return articles

#     def extract_review_details(self, articles, movie_url, limit=None):
#         """
#         Extracts review details from the loaded review articles.

#         Args:
#             articles (list): List of WebElement articles containing reviews.
#             movie_url (str): URL of the movie being scraped.
#             limit (int, optional): Maximum number of reviews to extract per movie. Defaults to 200.

#         Returns:
#             list: A list of extracted review dictionaries.
#         """
#         extracted_reviews = []
#         if limit:
#             extraction_articles = articles[:limit]
#         else:
#             extraction_articles = articles

#         for article in tqdm(extraction_articles,desc="extracting review details"):
#             try:
#                 review_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--rating').text
#             except NoSuchElementException:
#                 review_star = None

#             try:
#                 max_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--maxRating').text[1:]
#             except NoSuchElementException:
#                 max_star = None

#             try:
#                 review_title = article.find_element(By.CLASS_NAME, 'ipc-title__text').text
#             except NoSuchElementException:
#                 review_title = None

#             try:
#                 review_text = article.find_element(By.CLASS_NAME, "ipc-overflowText--children").text
#             except NoSuchElementException:
#                 review_text = None

#             extracted_reviews.append({
#                 "movie_url": movie_url,
#                 "review_star": review_star,
#                 "max_star": max_star,
#                 "review_title": review_title,
#                 "review_text": review_text
#             })

#         return extracted_reviews

#     def scrape_all_movies(self):
#         """
#         Scrapes reviews for all movies in the movie_urls list.
#         """
#         # for url in tqdm(self.movie_urls, desc="Processing movies"):
#         logger.info(f"Total movies - {len(self.movie_urls)}")
#         cnt = 1
#         for url in self.movie_urls:
#             logger.info(f"Starting - {cnt}")
#             logger.info(f"Scraping reviews for: {url}")
#             cnt+=1

#             reviews_page = self.get_reviews_page(url)
#             if not reviews_page:
#                 continue

#             self.driver.get(reviews_page)
#             time.sleep(3)

#             reviews_count = self.get_total_reviews_count()
#             if reviews_count == 0:
#                 continue

#             print(f"Total reviews: {reviews_count}")
#             articles = self.load_all_reviews(reviews_count)

#             print('extracting review details')
#             reviews_data = self.extract_review_details(articles, url)
#             self.all_reviews.extend(reviews_data)

#     def save_to_csv(self, filename="imdb_reviews.csv"):
#         """
#         Saves the scraped data to a CSV file.

#         Args:
#             filename (str, optional): Output CSV file name. Defaults to "imdb_reviews.csv".
#         """
#         df = pd.DataFrame(self.all_reviews)
#         df.to_csv(filename, index=False)
#         print(f"Scraping completed! Data saved to {filename}")

#     def close_driver(self):
#         """Closes the WebDriver."""
#         self.driver.quit()







In [4]:
# import time
# import re
# import json
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.common.exceptions import NoSuchElementException
# from tqdm import tqdm
# import os

# class IMDbReviewScraper:
#     """A scraper to extract user reviews from multiple IMDb movie pages."""

#     def __init__(self, movie_urls, progress_file="progress.json", output_file="imdb_reviews.csv"):
#         """
#         Initializes the IMDbReviewScraper.

#         Args:
#             movie_urls (list): List of IMDb movie URLs to scrape.
#             progress_file (str): Path to the progress log file.
#             output_file (str): Path to the output CSV file.
#         """
#         self.movie_urls = movie_urls
#         self.progress_file = progress_file
#         self.output_file = output_file
#         self.driver = webdriver.Chrome()
#         self.all_reviews = []
#         self.completed_urls = self.load_progress()

#     def load_progress(self):
#         """
#         Loads progress from the progress file.

#         Returns:
#             set: A set of completed movie URLs.
#         """
#         if os.path.exists(self.progress_file):
#             with open(self.progress_file, "r") as file:
#                 return set(json.load(file))
#         return set()

#     def save_progress(self, url):
#         """
#         Saves the completed movie URL to the progress file.

#         Args:
#             url (str): The URL of the completed movie.
#         """
#         self.completed_urls.add(url)
#         with open(self.progress_file, "w") as file:
#             json.dump(list(self.completed_urls), file)

#     def append_to_csv(self, reviews):
#         """
#         Appends scraped reviews to the output CSV file.

#         Args:
#             reviews (list): List of review dictionaries.
#         """
#         df = pd.DataFrame(reviews)
#         df.to_csv(self.output_file, mode='a', header=not os.path.exists(self.output_file), index=False)

#     def get_reviews_page(self, url):
#         """Navigates to the IMDb movie reviews page."""
#         self.driver.get(url)
#         time.sleep(3)
#         try:
#             click_reviews = self.driver.find_element(By.XPATH, "//div[@data-testid='reviews-header']")
#             return click_reviews.find_element(By.TAG_NAME, 'a').get_attribute('href')
#         except NoSuchElementException:
#             print(f"Reviews link not found for {url}")
#             return None

#     def get_total_reviews_count(self):
#         """Extracts the total number of reviews available for the movie."""
#         try:
#             reviews_element = self.driver.find_element(By.XPATH, "//div[@data-testid='tturv-total-reviews']")
#             return int(re.sub(r"[^\d]", "", reviews_element.text))
#         except NoSuchElementException:
#             print("Could not retrieve total reviews count.")
#             return 0

#     def click_show_all_button(self):
#         """Clicks the 'Show All' button to load more reviews."""
#         try:
#             span_element = self.driver.find_element(By.XPATH, "//span[contains(@class, 'chained-see-more-button')]")
#             button = span_element.find_element(By.TAG_NAME, "button")
#             ActionChains(self.driver).move_to_element(button).click().perform()
#         except Exception:
#             pass

#     def load_all_reviews(self, reviews_count, max_wait_time=600, polling_interval=1.5):
#         """Loads all available reviews by scrolling and clicking 'Show All'."""
#         start_time = time.time()
#         self.click_show_all_button()
#         while True:
#             articles = self.driver.find_elements(By.TAG_NAME, "article")
#             if len(articles) >= reviews_count or time.time() - start_time > max_wait_time:
#                 break
#             if articles:
#                 self.driver.execute_script("arguments[0].scrollIntoView();", articles[-1])
#             self.click_show_all_button()
#             time.sleep(polling_interval)
#         return articles

#     def extract_review_details(self, articles, movie_url):
#         """Extracts review details from the loaded review articles."""
#         extracted_reviews = []
#         for article in articles:
#             try:
#                 review_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--rating').text
#             except NoSuchElementException:
#                 review_star = None
#             try:
#                 max_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--maxRating').text[1:]
#             except NoSuchElementException:
#                 max_star = None
#             try:
#                 review_title = article.find_element(By.CLASS_NAME, 'ipc-title__text').text
#             except NoSuchElementException:
#                 review_title = None
#             try:
#                 review_text = article.find_element(By.CLASS_NAME, "ipc-overflowText--children").text
#             except NoSuchElementException:
#                 review_text = None
#             extracted_reviews.append({
#                 "movie_url": movie_url,
#                 "review_star": review_star,
#                 "max_star": max_star,
#                 "review_title": review_title,
#                 "review_text": review_text
#             })
#         return extracted_reviews

#     def scrape_all_movies(self):
#         """Scrapes reviews for all movies in the movie_urls list."""
#         remaining_urls = [url for url in self.movie_urls if url not in self.completed_urls]
#         for url in tqdm(remaining_urls, desc="Processing movies"):
#             print(f"Scraping reviews for: {url}")
#             reviews_page = self.get_reviews_page(url)
#             if not reviews_page:
#                 continue
#             self.driver.get(reviews_page)
#             time.sleep(3)
#             reviews_count = self.get_total_reviews_count()
#             if reviews_count == 0:
#                 continue
#             articles = self.load_all_reviews(reviews_count)
#             reviews_data = self.extract_review_details(articles, url)
#             self.append_to_csv(reviews_data)
#             self.save_progress(url)
#         print("Scraping completed!")

#     def close_driver(self):
#         """Closes the WebDriver."""
#         self.driver.quit()

# # Example usage:
# if __name__ == "__main__":
#     movie_urls = [
#         "https://www.imdb.com/title/tt10888594/",
#         "https://www.imdb.com/title/tt12345678/",
#         # Add more movie URLs
#     ]
#     scraper = IMDbReviewScraper(movie_urls)
#     scraper.scrape_all_movies()
#     scraper.close_driver()


In [5]:
import time
import re
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
import os

class IMDbReviewScraper:
    """A scraper to extract user reviews from multiple IMDb movie pages."""

    def __init__(self, movie_urls,movies_dict, progress_file="progress.json", output_file="imdb_reviews.csv"):
        """
        Initializes the IMDbReviewScraper.

        Args:
            movie_urls (list): List of IMDb movie URLs to scrape.
            progress_file (str): Path to the progress log file.
            output_file (str): Path to the output CSV file.
        """
        self.movie_urls = movie_urls
        self.movies_dict = movies_dict
        self.progress_file = progress_file
        self.output_file = output_file
        self.driver = webdriver.Chrome()
        self.all_reviews = []
        self.completed_urls = self.load_progress()

    def load_progress(self):
        """
        Loads progress from the progress file.

        Returns:
            set: A set of completed movie URLs.
        """
        if os.path.exists(self.progress_file):
            with open(self.progress_file, "r") as file:
                return set(json.load(file))
        return set()

    def save_progress(self, url):
        """
        Saves the completed movie URL to the progress file.

        Args:
            url (str): The URL of the completed movie.
        """
        self.completed_urls.add(url)
        with open(self.progress_file, "w") as file:
            json.dump(list(self.completed_urls), file)

    def append_to_csv(self, reviews):
        """
        Appends scraped reviews to the output CSV file.

        Args:
            reviews (list): List of review dictionaries.
        """
        df = pd.DataFrame(reviews)
        df.to_csv(self.output_file, mode='a', header=not os.path.exists(self.output_file), index=False)

    def get_reviews_page(self, url):
        """Navigates to the IMDb movie reviews page."""
        self.driver.get(url)
        try:
            # Wait until the reviews header is present and clickable
            click_reviews = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@data-testid='reviews-header']"))
            )
            return click_reviews.find_element(By.TAG_NAME, 'a').get_attribute('href')

        except NoSuchElementException:
            logger.info(f"Reviews link not found for {url}")
            return None

        except Exception as e:
            logger.error(f"An error occurred: {e}")
            return None
        # time.sleep(3)
        # try:
        #     click_reviews = self.driver.find_element(By.XPATH, "//div[@data-testid='reviews-header']")
        #     return click_reviews.find_element(By.TAG_NAME, 'a').get_attribute('href')
        # except NoSuchElementException:
        #     logger.info(f"Reviews link not found for {url}")
        #     return None

    def get_total_reviews_count(self):
        """Extracts the total number of reviews available for the movie."""
        try:
            reviews_element = self.driver.find_element(By.XPATH, "//div[@data-testid='tturv-total-reviews']")
            return int(re.sub(r"[^\d]", "", reviews_element.text))
        except NoSuchElementException:
            logger.info("Could not retrieve total reviews count.")
            return 0

    # def click_show_all_button(self):
    #     """Clicks the 'Show All' button to load more reviews."""
    #     try:
    #         span_element = self.driver.find_element(By.XPATH, "//span[contains(@class, 'chained-see-more-button')]")
    #         button = span_element.find_element(By.TAG_NAME, "button")
    #         ActionChains(self.driver).move_to_element(button).click().perform()
    #     except Exception:
    #         pass
    
    # def click_show_all_button(self):
    #     """
    #     Clicks the 'Show All' button to load more reviews.
    #     """
    #     try:
    #         span_element = self.driver.find_element(By.XPATH, "//span[contains(@class, 'ipc-see-more')]")
    #         button = span_element.find_element(By.TAG_NAME, "button")
    #         if button.text == 'All':
    #             logger.info('clicking All')
    #             ActionChains(self.driver).move_to_element(button).click().perform()
    #     except Exception:
    #         pass
    
    # def click_show_all_button(self):
    #     """
    #     Clicks the 'All' button inside the pagination div to load all reviews.
    #     """
    #     try:
    #         # Find the pagination div
    #         pagination_div = self.driver.find_element(By.XPATH, "//div[@data-testid='tturv-pagination']")
            
    #         # Find the 'All' button inside the pagination div
    #         all_button = pagination_div.find_element(By.XPATH, ".//span[contains(@class, 'ipc-see-more')]/button[.//span[text()='All']]")
            
    #         if all_button.is_displayed() and all_button.is_enabled():
    #             logger.info("Clicking 'All' button")
    #             ActionChains(self.driver).move_to_element(all_button).click().perform()
    #         else:
    #             logger.warning("'All' button is not clickable")
        
    #     except Exception as e:
    #         # logger.error(f"Error clicking 'All' button: {e}")
    #         pass

    # def load_all_reviews(self, reviews_count, max_wait_time=600, polling_interval=1.5):
    #     """Loads all available reviews by scrolling and clicking 'Show All'."""
    #     start_time = time.time()
    #     self.click_show_all_button()
    #     while True:
    #         articles = self.driver.find_elements(By.TAG_NAME, "article")
    #         logger.info(f"extracted - {articles} review articles")
    #         if len(articles) >= reviews_count or time.time() - start_time > max_wait_time:
    #             break
    #         if articles:
    #             self.driver.execute_script("arguments[0].scrollIntoView();", articles[-1])
    #         # self.click_show_all_button()
    #         time.sleep(polling_interval)
    #     return articles
    
    # def click_show_all_button(self):
    #     """
    #     Clicks the 'All' button inside the pagination div to load all reviews.
    #     If the 'All' button is not clickable, clicks the 'more' button instead.
    #     """
    #     try:
    #         pagination_div = self.driver.find_element(By.XPATH, "//div[@data-testid='tturv-pagination']")
            
    #         try:
    #             all_button = pagination_div.find_element(By.XPATH, ".//span[contains(@class, 'ipc-see-more')]/button[.//span[text()='All']]")
    #             if all_button.is_displayed() and all_button.is_enabled():
    #                 logger.info("Clicking 'All' button")
    #                 ActionChains(self.driver).move_to_element(all_button).click().perform()
    #                 return
    #         except Exception:
    #             logger.warning("'All' button not found or not clickable")
            
    #         # Fallback: Click on the 'more' button if 'All' is not available
    #         more_buttons = pagination_div.find_elements(By.XPATH, ".//button[contains(@class, 'ipc-see-more__button') and contains(text(), 'more')]")
    #         if more_buttons:
    #             for button in more_buttons:
    #                 if button.is_displayed() and button.is_enabled():
    #                     logger.info("Clicking 'more' button")
    #                     ActionChains(self.driver).move_to_element(button).click().perform()
    #                     return
            
    #         # logger.warning("No clickable 'All' or 'more' button found")
            
        
    #     except Exception as e:
    #         # logger.error(f"Error clicking pagination buttons: {e}")
            
    #         pass
    
    def click_show_all_button(self):
        """
        Clicks the 'All' button inside the pagination div to load all reviews.
        If the 'All' button is not clickable, clicks the 'more' button instead.
        """
        try:
            pagination_div = self.driver.find_element(By.XPATH, "//div[@data-testid='tturv-pagination']")
            
            # Attempt to find and click the 'All' button
            try:
                all_button = pagination_div.find_element(By.XPATH, ".//span[contains(@class, 'ipc-see-more')]/button[.//span[text()='All']]")
                if all_button.is_displayed() and all_button.is_enabled():
                    logger.info("Clicking 'All' button")
                    ActionChains(self.driver).move_to_element(all_button).click().perform()
                    return
            except Exception:
                # logger.warning("'All' button not found or not clickable")
                pass
            
            # Attempt to find and click the 'more' button if 'All' is not available
            try:
                # Wait for the pagination div to be present
                pagination_div = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@data-testid='tturv-pagination']"))
                )

                # Locate the 'more' button inside the pagination div
                more_button = pagination_div.find_element(By.XPATH, ".//button[.//span[contains(text(), ' more')]]")

                # Ensure the button is visible and clickable
                if more_button.is_displayed() and more_button.is_enabled():
                    # logger.info("Clicking 'more' button")
                    
                    # Use ActionChains to ensure smooth interaction
                    ActionChains(self.driver).move_to_element(more_button).click().perform()
                    return
            except Exception as e:
                # logger.warning(f"'More' button not found or not clickable: {str(e)}")
                pass
            
            # logger.info("No clickable 'All' or 'more' button found, skipping action")
        
        except Exception as e:
            # logger.error(f"Error clicking pagination buttons: {e}")
            pass

    def load_all_reviews(self, reviews_count, max_wait_time=1200, polling_interval=1.5):
        """Loads all available reviews by scrolling and clicking 'Show All'."""
        start_time = time.time()
        self.click_show_all_button()
        stable_count = 0
        last_count = 0
        
        while True:
            articles = self.driver.find_elements(By.TAG_NAME, "article")
            # logger.info(f"extracted - {len(articles)} review articles")
            
            if len(articles) >= reviews_count or time.time() - start_time > max_wait_time:
                break
            
            if len(articles) == last_count:
                stable_count += 1
            else:
                stable_count = 0
            
            if stable_count >= 5:
                logger.info("Article count has remained the same for 5 consecutive checks. Stopping load.")
                break
            
            last_count = len(articles)
            
            if articles:
                self.driver.execute_script("arguments[0].scrollIntoView();", articles[-1])
            
            time.sleep(polling_interval)
            self.click_show_all_button()
        
        return articles
    
    def extract_review_details(self, articles, movie_url):
        """Extracts review details from the loaded review articles."""
        extracted_reviews = []
        for article in tqdm(articles,desc='extracting review details'):
            try:
                review_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--rating').text
            except NoSuchElementException:
                review_star = None
            try:
                max_star = article.find_element(By.CLASS_NAME, 'ipc-rating-star--maxRating').text[1:]
            except NoSuchElementException:
                max_star = None
            try:
                review_title = article.find_element(By.CLASS_NAME, 'ipc-title__text').text
            except NoSuchElementException:
                review_title = None
            try:
                review_text = article.find_element(By.CLASS_NAME, "ipc-overflowText--children").text
            except NoSuchElementException:
                review_text = None
            extracted_reviews.append({
                "movie_url": movie_url,
                "review_star": review_star,
                "max_star": max_star,
                "review_title": review_title,
                "review_text": review_text
            })
        return extracted_reviews

    def scrape_all_movies(self):
        """Scrapes reviews for all movies in the movie_urls list."""
        remaining_urls = [url for url in self.movie_urls if url not in self.completed_urls]
        for idx, url in enumerate(remaining_urls, start=1):
            logger.info(f"Starting - {idx} Scraping reviews for: {self.movies_dict[url]}")
            reviews_page = self.get_reviews_page(url)
            if not reviews_page:
                continue
            self.driver.get(reviews_page)
            time.sleep(3)
            reviews_count = self.get_total_reviews_count()
            logger.info(f"total reviews - {reviews_count}")
            if reviews_count == 0:
                continue
            articles = self.load_all_reviews(reviews_count)
            reviews_data = self.extract_review_details(articles, url)
            self.append_to_csv(reviews_data)
            self.save_progress(url)
            if idx % 1000 == 0:
                user_input = input("Processed 1000 movies. Do you want to continue? (yes/no): ")
                if user_input.lower() != 'yes':
                    logger.info("Stopping as per user request.")
                    break
        logger.info("Scraping completed!")

    def close_driver(self):
        """Closes the WebDriver."""
        self.driver.quit()

In [6]:
# from time import time

In [7]:
import pandas as pd

In [8]:
movies_index = pd.read_csv(r"C:\Users\Harish Raju\Desktop\projects\IMDB\web_scrapper\data\Movies_index_final.csv")
movie_urls = movies_index['IMDb Link'].to_list()

In [9]:
movies_index.head()

,Title,IMDb Link,category
0,Radhe,https://www.imdb.com/title/tt10888594/?ref_=sr...,1_3
1,Adipurush,https://www.imdb.com/title/tt12915716/?ref_=sr...,1_3
2,Meet the Spartans,https://www.imdb.com/title/tt1073498/?ref_=sr_i_3,1_3
3,Epic Movie,https://www.imdb.com/title/tt0799949/?ref_=sr_i_4,1_3
4,Sadak 2,https://www.imdb.com/title/tt7886848/?ref_=sr_i_5,1_3


In [10]:
movies_dict = dict(zip(movies_index['IMDb Link'],movies_index['Title']))

In [11]:
# # Initialize and run scraper
# scraper = IMDbReviewScraper(movie_urls[:5])
# scraper.scrape_all_movies()
# scraper.save_to_csv()
# scraper.close_driver()

In [12]:
scraper = IMDbReviewScraper(movie_urls,movies_dict)
scraper.scrape_all_movies()
scraper.close_driver()

2025-03-18 09:46:22 - [INFO] - (1551185206.py:303) - Starting - 1 Scraping reviews for: Decrepit Crescendo
2025-03-18 09:46:25 - [INFO] - (1551185206.py:76) - Reviews link not found for https://www.imdb.com/title/tt2679334/?ref_=sr_i_271
2025-03-18 09:46:25 - [INFO] - (1551185206.py:303) - Starting - 2 Scraping reviews for: We Love Bad Boys
2025-03-18 09:46:29 - [INFO] - (1551185206.py:76) - Reviews link not found for https://www.imdb.com/title/tt31451148/?ref_=sr_i_346
2025-03-18 09:46:29 - [INFO] - (1551185206.py:303) - Starting - 3 Scraping reviews for: Keratin
2025-03-18 09:46:32 - [INFO] - (1551185206.py:76) - Reviews link not found for https://www.imdb.com/title/tt12664876/?ref_=sr_i_363
2025-03-18 09:46:32 - [INFO] - (1551185206.py:303) - Starting - 4 Scraping reviews for: Justin Bieber: Rise to Fame
2025-03-18 09:46:35 - [INFO] - (1551185206.py:76) - Reviews link not found for https://www.imdb.com/title/tt1852818/?ref_=sr_i_366
2025-03-18 09:46:35 - [INFO] - (1551185206.py:303)

In [4]:
import pandas as pd

In [13]:
df = pd.read_csv(r'C:\Users\Harish Raju\Desktop\projects\IMDB\imdb_reviews.csv')

In [6]:
df

,movie_url,review_star,max_star,review_title,review_text
0,https://www.imdb.com/title/tt10888594/?ref_=sr...,1.0,10.0,Donate money for Corona patients rather to was...,Salman has competition from himself to make wo...
1,https://www.imdb.com/title/tt10888594/?ref_=sr...,1.0,10.0,Trash of the year,"Storyline Trash, Acting of Salman khan Trash, ..."
2,https://www.imdb.com/title/tt10888594/?ref_=sr...,1.0,10.0,As always.,This is the best movie if you want to nit watc...
3,https://www.imdb.com/title/tt10888594/?ref_=sr...,1.0,10.0,Worst movie of all the time,Salman Khan needs to take retirement. This mov...
4,https://www.imdb.com/title/tt10888594/?ref_=sr...,1.0,10.0,Maha Bakwassss...,I would rather prefer watching TVF webseries a...
...,...,...,...,...,...
38901,https://www.imdb.com/title/tt6836772/?ref_=sr_...,3.0,10.0,"""Tell them I was a hero""",NaN
38902,https://www.imdb.com/title/tt6836772/?ref_=sr_...,4.0,10.0,"Band of brothers, meet's the longest day!",I think the idea was to keep this in mind for ...
38903,https://www.imdb.com/title/tt6836772/?ref_=sr_...,NaN,NaN,A Disgrace to This Who Served and Sacrificed,This is a bad movie. I won't even go into how ...
38904,https://www.imdb.com/title/tt6836772/?ref_=sr_...,NaN,NaN,Horrible show and complete waste of time,"Everything has gone wrong here;the plot, scree..."


In [7]:
df['movie_url'].value_counts()

movie_url
https://www.imdb.com/title/tt10888594/?ref_=sr_i_1      2599
https://www.imdb.com/title/tt1754700/?ref_=sr_i_24      1105
https://www.imdb.com/title/tt10171746/?ref_=sr_i_162     967
https://www.imdb.com/title/tt2574698/?ref_=sr_i_14       896
https://www.imdb.com/title/tt12915716/?ref_=sr_i_2       754
                                                        ... 
https://www.imdb.com/title/tt7535868/?ref_=sr_i_143        4
https://www.imdb.com/title/tt1570619/?ref_=sr_i_165        3
https://www.imdb.com/title/tt2180158/?ref_=sr_i_229        3
https://www.imdb.com/title/tt0953989/?ref_=sr_i_142        2
https://www.imdb.com/title/tt0494501/?ref_=sr_i_114        2
Name: count, Length: 211, dtype: int64

In [8]:
df['review_star'].value_counts()

review_star
1.0     20743
10.0     4773
2.0      2762
8.0      1595
3.0      1499
9.0      1274
4.0      1015
5.0      1001
7.0       995
6.0       792
Name: count, dtype: int64

In [ ]:
# url = 'https://www.imdb.com/title/tt10888594/?ref_=sr_i_1'

In [ ]:
# driver = webdriver.Chrome()

# reviews_link = url
# # got movie title and link

# driver.get(reviews_link)
# time.sleep(3)
# click_reviews = driver.find_element(By.XPATH, "//div[@data-testid='reviews-header']")
# reviews_link = click_reviews.find_element(By.TAG_NAME,'a').get_attribute('href')
# driver.get(reviews_link)
# reviews_element = driver.find_element(By.XPATH, "//div[@data-testid='tturv-total-reviews']")
# reviews_text = reviews_element.text
# # Use regex to extract only the numeric part
# reviews_count = int(re.sub(r"[^\d]", "", reviews_text))

# print(reviews_count)
# def click_all_button():
#     #  click button to show all reviews
#     try:
#         # Locate the correct <span> first, then find the button inside it
#         span_element = driver.find_element(By.XPATH, "//span[contains(@class, 'chained-see-more-button')]")
#         button = span_element.find_element(By.TAG_NAME, "button")

#         if button:
#             # Click the button using ActionChains (sometimes needed for better interaction)
#             ActionChains(driver).move_to_element(button).click().perform()
        
#             print("Correct 'All' button clicked successfully.")
#     except Exception as e:
#         print(f"Error: {e}")
# max_wait_time = 600  
# polling_interval = 1.5  # Check every 1.5 seconds
# target_count = reviews_count  # Expected number of articles

# start_time = time.time()

# click_all_button()

# while True:
#     # Count current number of loaded articles
#     articles = driver.find_elements(By.TAG_NAME, "article")
#     current_count = len(articles)

#     print(f"Loaded articles: {current_count}")

#     # If we reach the target, break the loop
#     if current_count >= target_count:
#         print("All articles loaded!")
#         break

#     # Stop waiting if max wait time is exceeded
#     if time.time() - start_time > max_wait_time:
#         print("Timeout reached. Extracting available articles.")
#         break
    

#     if articles:
#         last_article = articles[-1]  # Get the last article
#         driver.execute_script("arguments[0].scrollIntoView();", last_article)
#         # print("Scrolled to the last article.")
#     else:
#         # print("No articles found.")
#         pass
    
#     click_all_button()
    
#     time.sleep(polling_interval)  # Wait before checking again


In [ ]:
# sample_articles = articles[:100]
# review_stars = []
# max_stars = []
# review_titles = []
# review_texts = []



# for article in tqdm(sample_articles):
    
#     review_star = None
#     try:
#         review_star =  article.find_element(By.CLASS_NAME,'ipc-rating-star--rating').text
#     except:
#         review_star = None
        
#     review_stars.append(review_star)
        
    
        
#     max_star = None
#     try:
#         max_star =  article.find_element(By.CLASS_NAME,'ipc-rating-star--maxRating').text[1:]
#     except:
#         max_star = None
#     max_stars.append(max_star)
        
#     review_title = None
#     try:
#         review_title =  article.find_element(By.CLASS_NAME,'ipc-title__text').text
#     except:
#         review_title = None
    
#     review_titles.append(review_title)
        

    
#     # review_text = None

#     # try:
#     #     # Check if Spoiler button exists
#     #     spoiler_button = article.find_element(By.XPATH, "//button[contains(@class, 'review-spoiler-button')]")
#     #     if spoiler_button:
#     #         review_text = None  # Set to None if spoiler button is found
#     #     else:
#     #         review_text = article.find_element(By.XPATH, "//div[@data-testid='review-overflow']").text
#     # except NoSuchElementException:
#     #     # If no spoiler button is found, try extracting review
#     #     try:
#     #         review_text = article.find_element(By.XPATH, "//div[@data-testid='review-overflow']").text
#     #     except NoSuchElementException:
#     #         review_text = None  # Set to None if review is also not found
    
    
#     review_text = None
#     try:
#         review_text =  article.find_element(By.CLASS_NAME,"ipc-overflowText--children").text
#     except:
#         review_text = None
            
#     review_texts.append(review_text)
        
    